In [142]:
import pymssql
import requests
import pandas as pd

In [151]:
def SQL_INSERT1():
    
    #get films
    
    query='''--Отбор фильмов для доливки в файл MovieList
    insert into [MovieDB].[dbo].[test-MovieList] ([MovieId],[MovieName],[MovieNameUkr],[MovieId-2],[Now],[imdb ])
    select
    distinct
    [sms_ws].[dbo].[KIS_Reference265]._Code as 'MovieId', 
    cast([sms_ws].[dbo].[KIS_Reference265]._Fld4927 as nvarchar) as 'MovieName',
    cast([sms_ws].[dbo].[KIS_Reference265].[_Fld13529] as nvarchar)  as 'MovieNameUkr', 
    [sms_ws].[dbo].[KIS_Reference265]._Fld13528 as 'MovieId-2',
    0 as 'Now',
    /*max(case when dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) >= getdate() 
          and dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) <= dateadd(month,1,getdate()) then 1 else 0 end) as 'Now',*/
    [sms_ws].[dbo].[KIS_Reference265].[_Fld17288] as 'imdb'
    from KIS_Reference265
    left join [sms_ws].[dbo].[KIS_Document13058] on [sms_ws].[dbo].[KIS_Document13058].[_Fld13219RRef]=[sms_ws].[dbo].[KIS_Reference265].[_IDRRef] 
    left join (--Развернуть таблицу и обьеденить поля
          select [Тип] ,( select [Имя]+',' as 'data()' 
                  from (select
                      [KIS_Reference265]._IDRRef as 'Тип',
                      KIS_Reference265_VT17304._Fld17307 as 'Имя'
                      from [KIS_Reference265]
                      left join KIS_Reference265_VT17304 on KIS_Reference265_VT17304._Reference265_IDRRef = [KIS_Reference265]._IDRRef
                      ) t2 where t1.[Тип]=t2.[Тип] for xml path('') 
                  ) as 'Tags'
          from (select
             [KIS_Reference265]._IDRRef as 'Тип',
             KIS_Reference265_VT17304._Fld17307 as 'Имя'
             from [KIS_Reference265]
             left join KIS_Reference265_VT17304 on KIS_Reference265_VT17304._Reference265_IDRRef = [KIS_Reference265]._IDRRef
             ) t1
          group by [Тип]
          ) as Tags on Tags.[Тип] = [KIS_Reference265]._IDRRef    
    where dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) >= dateadd(day,-5,getdate())
    and [sms_ws].[dbo].[KIS_Reference265].[_Fld17288] like ('tt%')
    and [sms_ws].[dbo].[KIS_Reference265]._Fld13528 not in (select [MovieId-2] from [MovieDB].[dbo].[test-MovieList])
    and [sms_ws].[dbo].[KIS_Reference265].[_Fld17288] not in (select [imdb] from [MovieDB].[dbo].[test-MovieList])
    and case when Tags.Tags = '' then '' else LEFT(Tags.Tags, LEN(Tags.Tags) - 1) end not like ('%КіноМама%')
    and case when Tags.Tags = '' then '' else LEFT(Tags.Tags, LEN(Tags.Tags) - 1) end not like ('%In English%')
    and case when Tags.Tags = '' then '' else LEFT(Tags.Tags, LEN(Tags.Tags) - 1) end not like ('%Допрем''єрний показ%')
    '''
    
    conn = pymssql.connect(
        host='10.128.150.49',
        user=r'InsertToMovieDB',
        password='Pgz%sE9w5SR4',
        database='sms_ws', 
    charset='cp1251')

    cur = conn.cursor()
    
    try:
        cur.execute(query)
        conn.commit()
    finally:
        conn.close()


In [152]:
def SQL_INSERT2(): 
    query='''
    
        -----------------------------------------------------------------------------------------------------------------------------

    --Присваивается всем фильмам статус НЕ в прокате

    update [MovieDB].[dbo].[test-MovieList]
    set [MovieDB].[dbo].[test-MovieList].[Now] = 0
    where [MovieDB].[dbo].[test-MovieList].[Now] = 1
   
    '''
    conn = pymssql.connect(
        host='10.128.150.49',
        user=r'InsertToMovieDB',
        password='Pgz%sE9w5SR4',
        database='sms_ws', 
    charset='cp1251')
    
    cur = conn.cursor()
    try:
        cur.execute(query)
    finally:
        conn.close()
#     a = pd.read_sql(query,conn)


In [153]:
def SQL_INSERT3(): 
#     try:
    query='''
    update [MovieDB].[dbo].[test-MovieList]
    set [MovieDB].[dbo].[test-MovieList].[Now] = 1
    where [MovieDB].[dbo].[test-MovieList].[MovieId-2] in 
          (select 
         distinct
         [sms_ws].[dbo].[KIS_Reference265]._Fld13528 
         from [sms_ws].[dbo].[KIS_Reference265]
         left join [sms_ws].[dbo].[KIS_Document13058] on [sms_ws].[dbo].[KIS_Document13058].[_Fld13219RRef]=[sms_ws].[dbo].[KIS_Reference265].[_IDRRef] 
         where dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) >= getdate() 
             and dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) <= dateadd(month,1,getdate())
           )
    '''
    conn = pymssql.connect(
        host='10.128.150.49',
        user=r'InsertToMovieDB',
        password='Pgz%sE9w5SR4',
        database='MovieDB', 
    charset='cp1251')

    cur = conn.cursor()
    try:
        cur.execute(query)
    finally:
        conn.close()
#     except:
#         print('Error')
#         conn.close()
#     a = pd.read_sql(query,conn)


In [154]:
def SQL_ML()   : 
    
    query1='SELECT * from [test-MovieList]'
    conn = pymssql.connect(
        host='10.128.150.49',
        user=r'InsertToMovieDB',
        password='Pgz%sE9w5SR4',
        database='MovieDB', 
    charset='cp1251')
    
    df_movie_list=pd.read_sql(query1,conn)
    return df_movie_list

In [146]:
# def SQL_GET_NEW(): 
#     query='''--Отбор фильмов для доливки в файл MovieList
#     select
#     distinct
#     [sms_ws].[dbo].[KIS_Reference265]._Code as 'MovieId', 
#     cast([sms_ws].[dbo].[KIS_Reference265]._Fld4927 as nvarchar) as 'MovieName',
#     cast([sms_ws].[dbo].[KIS_Reference265].[_Fld13529] as nvarchar)  as 'MovieNameUkr', 
#     [sms_ws].[dbo].[KIS_Reference265]._Fld13528 as 'MovieId-2',
#     0 as 'Now',
#     /*max(case when dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) >= getdate() 
#           and dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) <= dateadd(month,1,getdate()) then 1 else 0 end) as 'Now',*/
#     [sms_ws].[dbo].[KIS_Reference265].[_Fld17288] as 'imdb'
#     from KIS_Reference265
#     left join [sms_ws].[dbo].[KIS_Document13058] on [sms_ws].[dbo].[KIS_Document13058].[_Fld13219RRef]=[sms_ws].[dbo].[KIS_Reference265].[_IDRRef] 
#     left join (--Развернуть таблицу и обьеденить поля
#           select [Тип] ,( select [Имя]+',' as 'data()' 
#                   from (select
#                       [KIS_Reference265]._IDRRef as 'Тип',
#                       KIS_Reference265_VT17304._Fld17307 as 'Имя'
#                       from [KIS_Reference265]
#                       left join KIS_Reference265_VT17304 on KIS_Reference265_VT17304._Reference265_IDRRef = [KIS_Reference265]._IDRRef
#                       ) t2 where t1.[Тип]=t2.[Тип] for xml path('') 
#                   ) as 'Tags'
#           from (select
#              [KIS_Reference265]._IDRRef as 'Тип',
#              KIS_Reference265_VT17304._Fld17307 as 'Имя'
#              from [KIS_Reference265]
#              left join KIS_Reference265_VT17304 on KIS_Reference265_VT17304._Reference265_IDRRef = [KIS_Reference265]._IDRRef
#              ) t1
#           group by [Тип]
#           ) as Tags on Tags.[Тип] = [KIS_Reference265]._IDRRef    
#     where dateadd(year,-2000,[sms_ws].[dbo].[KIS_Document13058]._Fld13203) >= dateadd(day,-5,getdate())
#     and [sms_ws].[dbo].[KIS_Reference265].[_Fld17288] like ('tt%')
#     and [sms_ws].[dbo].[KIS_Reference265]._Fld13528 not in (select [MovieId-2] from [MovieDB].[dbo].[test-MovieList])
#     and [sms_ws].[dbo].[KIS_Reference265].[_Fld17288] not in (select [imdb] from [MovieDB].[dbo].[test-MovieList])
#     and case when Tags.Tags = '' then '' else LEFT(Tags.Tags, LEN(Tags.Tags) - 1) end not like ('%КіноМама%')
#     and case when Tags.Tags = '' then '' else LEFT(Tags.Tags, LEN(Tags.Tags) - 1) end not like ('%In English%')
#     and case when Tags.Tags = '' then '' else LEFT(Tags.Tags, LEN(Tags.Tags) - 1) end not like ('%Допрем''єрний показ%')
#     '''
#     conn = pymssql.connect(
#         host='10.128.150.49',
#         user=r'InsertToMovieDB',
#         password='Pgz%sE9w5SR4',
#         database='sms_ws', 
#     charset='cp1251')
    
#     try:
#         df = pd.read_sql(query,conn)
#     finally:
#         conn.close()
#     return df

In [147]:
def main():
    
    SQL_INSERT1()
    print('1')
    SQL_INSERT2()
    print('2')
    SQL_INSERT3()
    print('3')
#     df = SQL_ML() 
    
#     return None
    
    TOKEN = 'YnJvd3Nlcjo='
    params = {

        'scope': 'ui',
        'username': 'Sasha',
        'password': 'qwerty',
        'grant_type': 'password'
    }
    print('4')
    r = requests.post('http://192.168.166.200:8080/uaa/oauth/token', params=params, headers={'Authorization': 'Basic %s' %  TOKEN})
    
    response = requests.get('http://192.168.166.200:8080/statistics/current', headers={'Authorization':'Bearer ' + r.json()['access_token']})
    all_films_postman = pd.DataFrame(response.json())
    meta_info = SQL_ML()
    
    
    print('5')
    for i in df.imdb:
        data = {
         "imdbId":i
        }
        a = requests.post('http://192.168.166.200:8080/statistics/current',json=data, headers={'Authorization':'Bearer ' + r.json()['access_token']})


In [148]:
main()

1
2
3
4


ConnectionError: HTTPConnectionPool(host='192.168.166.200', port=8080): Max retries exceeded with url: /uaa/oauth/token?scope=ui&username=Sasha&password=qwerty&grant_type=password (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002305E2BD6D8>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))

In [ ]:
meta_info = SQL_ML()

In [155]:
r = requests.post('http://192.168.166.200:8080/uaa/oauth/token', params=params, headers={'Authorization': 'Basic %s' %  TOKEN})
response = requests.get('http://192.168.166.200:8080/statistics/current', headers={'Authorization':'Bearer ' + r.json()['access_token']})
df = pd.DataFrame(response.json())


ConnectionError: HTTPConnectionPool(host='192.168.166.200', port=8080): Max retries exceeded with url: /uaa/oauth/token?scope=ui&username=Sasha&password=qwerty&grant_type=password (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023059CA9390>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))